<a href="https://colab.research.google.com/github/NancWN/ScrumBusters/blob/main/KeywordSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

What can it do?:
it takes a predefined list of keywords which is divided between pump and no_pump words. it uses the list to search trough chat data. if a match is found, the group name, the word and label is written down in a seperate list.

Requirements:
1. upload a list with Keywords you want to search for
2. provide chat data from telegram groups in CSV format
3. adjust the path in the code

Lets go! :)

In [ ]:
import csv
import pandas as pd 

#List of Keywords we want to search
lists=pd.read_csv("keywordList.csv", sep=';')
pump_words=lists["pump"]
no_pump_words=lists["no_pump"]

#telegram chat export (csv-file)
searchlists=pd.read_csv("chatdata.csv", sep=';',na_values = ['no info', '.'])



#searching for a keyword in our telegram chat export
def main(keyword):
    count=-1
    for i in range(len(searchlists)):
        count=count+1
        text=searchlists.iloc[count,1].lower()
        words = text.split(' ') 
        for word in words:
            if word==keyword:
                result=word
                group=searchlists.iloc[count,0]
                pump=labeling(result)
                write_to_csv([group, result, pump])
       
    
#lableling our found words -> here labeling can be extended                    
def labeling(word):
    count=-1
    label="unknown"
    for j in pump_words:
        count=count+1
        if word == pump_words.iloc[count]: 
            label=True
            break
    
    count=-1
    for k in no_pump_words:
        count=count+1
        if word == no_pump_words.iloc[count]: 
            label=False 
            break

    return label   
    
    
#going through our whole list of possible keywords
for keyword in pump_words: 
    main(keyword)
         
for keyword in no_pump_words: 
    main(keyword)

In [ ]:
def write_to_csv(list_input):
    # The scraped info will be written to a CSV here.
    try:
        with open("labelingKeywords.csv", "a") as fopen:  # Open the csv file.
            csv_writer = csv.writer(fopen)
            csv_writer.writerow(list_input)
    except:
        return False